In [ ]:
# verify_promela("./Exciercies/ex_1a.pml")

In [3]:
import re
def is_pymol_file(path: str) -> bool:

    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        for line in f:
            l = line.strip().lower()

            # Bỏ dòng trống và comment
            if not l or l.startswith("#"):
                continue

            # PyMOL command chắc chắn
            if l.startswith(("load ", "show ", "hide ", "color ", "set ","COMPND ","ATOM ")):
                return True

            # Nếu gặp code kiểu Promela trước → không phải PyMOL
            if any(k in l for k in ["proctype", "active", "init", "chan", "assert"]):
                return False


    return False

In [4]:
def check_promela_file(file_path):
    """
    Kiểm tra file có phải Promela (.pml) dựa trên từ khóa đặc trưng.
    Trả về True nếu đúng Promela, False nếu không.
    """
    promela_keywords = ["proctype", "init", "chan", "mtype"]

    try:
        with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
            content = f.read()

        # Chuyển về lowercase để kiểm tra
        content_lower = content.lower()

        # Nếu có ít nhất 1 từ khóa Promela → coi là Promela
        if any(keyword in content_lower for keyword in promela_keywords):
            # print(f"[+] File '{file_path}' có vẻ là **Promela (.pml)**.")
            return True
        else:
            # print(f"[-] File '{file_path}' không phải Promela.")
            return False

    except Exception as e:
        print(f"[!] Lỗi khi đọc file '{file_path}': {e}")
        return False

In [7]:
def is_xml(path: str) -> bool:

    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        for line in f:
            l = line.strip().lower()

            # Bỏ dòng trống và comment
            if not l or l.startswith("#"):
                continue
            if "http://ufal.mff.cuni.cz/pdt/pml/" in l:
                return True
            # PyMOL command chắc chắn
            lstLoc = ["<head>","</head>","<body>","<html>","xml","<Script>"]
            for word in lstLoc:
                if word in l:
                    return True
                    

            


    return False

In [11]:
import subprocess

# ===== CONFIG CHUẨN =====
PML_WIN_PATH = r"C:\Users\minhc\Downloads\spin651_windows64.exe\Exciercies\hanoi2.pm"
SPIN_LINUX = "/root/spin651_linux64"
# =======================

def win_to_wsl(path: str) -> str:
    drive = path[0].lower()
    rest = path[2:].replace("\\", "/")
    return f"/mnt/{drive}{rest}"

def run_wsl(cmd: str, check=True):
    return subprocess.run(
        ["wsl", "bash", "-lc", cmd],
        capture_output=True,
        text=True,
        check=check
    )

def verify_promela():
    pml = win_to_wsl(PML_WIN_PATH)
    print("🚀 Verifying with spin651_linux64")
    print("   PML:", pml)
    print("   SPIN:", SPIN_LINUX)
    
    # 0. kiểm tra file tồn tại
    run_wsl(f"ls '{pml}'")

    # 1. sinh pan.c
    run_wsl(f"{SPIN_LINUX} -a '{pml}'")

    # 2. compile verifier
    run_wsl("gcc -o pan pan.c")

    # 3. chạy verification
    result = run_wsl("./pan -a", check=False)

    print("====== SPIN OUTPUT ======")
    print(result.stdout)
    # print(is_pymol_file(pml))
    if "errors: 0" in result.stdout:
        print("✅ VERIFIED (no errors)")
    else:
        print("❌ LOGIC ERROR (see output above)")
    
if __name__ == "__main__":
    verify_promela()


🚀 Verifying with spin651_linux64
   PML: /mnt/c/Users/minhc/Downloads/spin651_windows64.exe/Exciercies/hanoi2.pm
   SPIN: /root/spin651_linux64
====== SPIN OUTPUT ======
error: max search depth too small

(Spin Version 6.5.1 -- 20 December 2019)
	+ Partial Order Reduction

Full statespace search for:
	never claim         	- (none specified)
	assertion violations	+
	acceptance   cycles 	- (not selected)
	invalid end states	+

State-vector 108 byte, depth reached 9999, errors: 0
    32708 states, stored
    65412 states, matched
    98120 transitions (= stored+matched)
        5 atomic steps
hash conflicts:        69 (resolved)

Stats on memory usage (in Megabytes):
    4.242	equivalent memory usage for states (stored*(State-vector + overhead))
    3.058	actual memory usage for states (compression: 72.08%)
         	state-vector as stored = 70 byte + 28 byte overhead
  128.000	memory used for hash table (-w24)
    0.534	memory used for DFS stack (-m10000)
    1.143	other (proc and chan s

In [13]:
import subprocess
import os

lstErr = list()
# ===== CONFIG =====
PML_WIN_DIR = r"C:\Users\minhc\Downloads\spin651_windows64.exe\dataPromela\content\promela_source_files"
SPIN_LINUX = "/root/spin651_linux64"
# ==================

def win_to_wsl(path: str) -> str:
    drive = path[0].lower()
    rest = path[2:].replace("\\", "/")
    return f"/mnt/{drive}{rest}"

def run_wsl(cmd: str, check=True):
    return subprocess.run(
        ["wsl", "bash", "-lc", cmd],
        capture_output=True,
        text=True,
        check=check
    )

def verify_one_file(pml_wsl: str):
    print(f"\n🚀 Verifying: {pml_wsl}")

    # Generate pan.c
    run_wsl(f"{SPIN_LINUX} -a '{pml_wsl}'")

    # Compile
    run_wsl("gcc -o pan pan.c")

    # Run verification
    result = run_wsl("./pan -a", check=False)

    if "errors: 0" in result.stdout:
        print("✅ VERIFIED (no errors)")
    else:
        print("❌ ERROR FOUND")
        for line in result.stdout.splitlines():
            if "error" in line.lower() or "assert" in line.lower():
                print("   ", line)

def verify_all():
    global lstErr
    print(f"📂 Scanning folder:")
    print(f"   {PML_WIN_DIR}")

    for fname in sorted(os.listdir(PML_WIN_DIR)):
        if fname.lower().endswith(".pml"):
            win_path = os.path.join(PML_WIN_DIR, fname)
            wsl_path = win_to_wsl(win_path)

            try:
                verify_one_file(wsl_path)
            except subprocess.CalledProcessError as e:
                print(f"💥 FAILED: {fname}")
                lstErr.append(fname)
                print(e.stderr)

# if __name__ == "__main__":
#     verify_all()


In [15]:
import re

def is_promela_file(path: str) -> bool:
    try:
        with open(path, "r", encoding="utf-8", errors="ignore") as f:
            lines = f.readlines()
    except Exception:
        return False

    text = "".join(lines).lower()

    # Promela keywords (word boundary)
    promela_patterns = [
        r"\bproctype\b",
        r"\binit\b",
        r"\bactive\b",
        r"\bchan\b",
        r"\bassert\b",
        r"\bltl\b",
        r"\bnever\b",
    ]

    # PyMOL commands (usually start of line)
    pymol_patterns = [
        r"^\s*load\s+",
        r"^\s*show\s+",
        r"^\s*hide\s+",
        r"^\s*color\s+",
        r"^\s*set\s+",
        r"^\s*cartoon\b",
        r"^\s*ribbon\b",
        r"^\s*sphere\b",
    ]

    # Nếu thấy PyMOL → loại
    for line in lines:
        for p in pymol_patterns:
            if re.search(p, line, re.IGNORECASE):
                return False

    # Phải có ít nhất 1 Promela keyword
    for p in promela_patterns:
        if re.search(p, text):
            return True

    return False


In [99]:
lstErr 

['C:\\Users\\minhc\\Downloads\\spin651_windows64.exe\\dataPromela\\content\\promela_source_files\\00013288540b0c3b6b781215a3fb1a30873935e2.pml',
 'C:\\Users\\minhc\\Downloads\\spin651_windows64.exe\\dataPromela\\content\\promela_source_files\\000645f93101f2436fac2ce428105cd162648208.pml',
 'C:\\Users\\minhc\\Downloads\\spin651_windows64.exe\\dataPromela\\content\\promela_source_files\\00093ac19486fdec3eaffb7ba7f0efd0c49e46ab.pml',
 'C:\\Users\\minhc\\Downloads\\spin651_windows64.exe\\dataPromela\\content\\promela_source_files\\000ae9de6ae39ba04f5920bc61f71e5e45baf87b.pml',
 'C:\\Users\\minhc\\Downloads\\spin651_windows64.exe\\dataPromela\\content\\promela_source_files\\000e151fb735ece5d601e5d5d54012695d1a0564.pml',
 'C:\\Users\\minhc\\Downloads\\spin651_windows64.exe\\dataPromela\\content\\promela_source_files\\000e67cd83274d02ae3827627873f4abce6e9591.pml',
 'C:\\Users\\minhc\\Downloads\\spin651_windows64.exe\\dataPromela\\content\\promela_source_files\\000eaf403ad0602378673bb2f5681f4f

In [101]:
import os
lstErr = list()
cnt_1 = 0 
for fname in sorted(os.listdir(PML_WIN_DIR)):
    if fname.lower().endswith(".pml"):
        win_path = os.path.join(PML_WIN_DIR, fname)
        if (is_xml(win_path) == True ) or (is_pymol_file(win_path) == True) or (check_promela_file(win_path) == False):
            lstErr.append(win_path)
        # if :
        #     lstErr.append(win_path)
            

In [102]:
cnt_1

0

In [103]:
len(lstErr)

1983

In [107]:
lstErr

['C:\\Users\\minhc\\Downloads\\spin651_windows64.exe\\dataPromela\\content\\promela_source_files\\000645f93101f2436fac2ce428105cd162648208.pml',
 'C:\\Users\\minhc\\Downloads\\spin651_windows64.exe\\dataPromela\\content\\promela_source_files\\00127488cc437d6a93d4a876f21d560501099e9f.pml',
 'C:\\Users\\minhc\\Downloads\\spin651_windows64.exe\\dataPromela\\content\\promela_source_files\\007b3196f7adb32dd737147f84f7c874ccb1edd0.pml',
 'C:\\Users\\minhc\\Downloads\\spin651_windows64.exe\\dataPromela\\content\\promela_source_files\\007f7450dc3ff48b9322681857bd95ff4a901bed.pml',
 'C:\\Users\\minhc\\Downloads\\spin651_windows64.exe\\dataPromela\\content\\promela_source_files\\00abeca94ba71a27a02c712fc06f90e80619630b.pml',
 'C:\\Users\\minhc\\Downloads\\spin651_windows64.exe\\dataPromela\\content\\promela_source_files\\00adfbad8f85480c97916eee5e07685c6cf7e4bf.pml',
 'C:\\Users\\minhc\\Downloads\\spin651_windows64.exe\\dataPromela\\content\\promela_source_files\\00d4fb7e6f4d17a2e3cb19b89a9a8ac9

In [109]:
DST_DIR = r"C:\Users\minhc\Downloads\spin651_windows64.exe\dataPromela\pymol_backup"
# os.makedirs(DST_DIR, exist_ok=True)

In [111]:
import shutil
for file_path in lstErr:
    if os.path.exists(file_path):
        shutil.move(file_path, DST_DIR)

In [113]:
# !pip install datasets

In [115]:
# from datasets import load_dataset

In [117]:
# !pip uninstall pyarrow
# !pip install --upgrade --force-reinstall pyarrow

In [119]:
import os
pyOML = dict()
for file in os.listdir("./dataPromela/pymol_backup"):
    # print(file.split(".")[0])
    pyOML[file.split(".")[0]] = 1

In [121]:
import json

# Đường dẫn file gốc và file kết quả
input_file = "clean_data.jsonl"
output_file = "clean_data_v2.jsonl"

# Điều kiện lọc: ví dụ chỉ giữ lại những dòng có trường "status" = "ok"
def is_valid(record):
    return record.get("status") == "ok"
cnt = 0 
with open(input_file, "r", encoding="utf-8") as infile, \
     open(output_file, "w", encoding="utf-8") as outfile:
    
    for line in infile:
        # try:
        record = json.loads(line)
        if (record["blob_id"] in pyOML ):
            continue
        cnt+=1
        # print(1)
            # print(record["blob_id"])
        outfile.write(json.dumps(record, ensure_ascii=False) + "\n")
        # except json.JSONDecodeError:
        #     # Nếu dòng không phải JSON hợp lệ thì bỏ qua
        #     continue


In [122]:
cnt

19598

In [139]:
def count_promela_tokens(file_path):
    """
    Đếm token trong file Promela.
    Token: từ, số, ký hiệu như ; {} []
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()

        # Loại bỏ comment (// hoặc /* ... */)
        content = re.sub(r'//.*', '', content)          # comment 1 dòng
        content = re.sub(r'/\*.*?\*/', '', content, flags=re.DOTALL)  # comment nhiều dòng

        # Tách token: từ + ký hiệu
        # Token là: chuỗi không phải space hoặc ký hiệu riêng, hoặc ký hiệu Promela riêng
        tokens = re.findall(r'[A-Za-z_][A-Za-z0-9_]*|[0-9]+|[{}();\[\]]', content)

        print(f"[+] File '{file_path}' có {len(tokens)} token.")
        return len(tokens)

    except Exception as e:
        print(f"[!] Lỗi khi đọc file '{file_path}': {e}")
        return 0

In [145]:
import os
lstNum = list()
max_ = 0
PML_WIN_DIR = r"C:\Users\minhc\Downloads\spin651_windows64.exe\dataPromela\content\promela_source_files"

for fname in sorted(os.listdir(PML_WIN_DIR)):
    fpath = os.path.join(PML_WIN_DIR, fname)
    if os.path.isfile(fpath):
        len_ = count_promela_tokens(fpath)
        lstNum.append(len_)
        max_ = max(max_,len_)

print(f"\n[+] Số token lớn nhất trong thư mục: {max_}")


[+] File 'C:\Users\minhc\Downloads\spin651_windows64.exe\dataPromela\content\promela_source_files\00013288540b0c3b6b781215a3fb1a30873935e2.pml' có 369 token.
[+] File 'C:\Users\minhc\Downloads\spin651_windows64.exe\dataPromela\content\promela_source_files\00093ac19486fdec3eaffb7ba7f0efd0c49e46ab.pml' có 82 token.
[+] File 'C:\Users\minhc\Downloads\spin651_windows64.exe\dataPromela\content\promela_source_files\000ae9de6ae39ba04f5920bc61f71e5e45baf87b.pml' có 2161 token.
[+] File 'C:\Users\minhc\Downloads\spin651_windows64.exe\dataPromela\content\promela_source_files\000e151fb735ece5d601e5d5d54012695d1a0564.pml' có 1335 token.
[+] File 'C:\Users\minhc\Downloads\spin651_windows64.exe\dataPromela\content\promela_source_files\000e67cd83274d02ae3827627873f4abce6e9591.pml' có 2459 token.
[+] File 'C:\Users\minhc\Downloads\spin651_windows64.exe\dataPromela\content\promela_source_files\000eaf403ad0602378673bb2f5681f4fdc40fa92.pml' có 453 token.
[+] File 'C:\Users\minhc\Downloads\spin651_windows

In [149]:
lstNum [lstNum < 8192 ]

TypeError: '<' not supported between instances of 'list' and 'int'

In [165]:
count = sum(1 for item in lstNum if item < 1024)

In [167]:
len(lstNum)

19598

In [169]:
count

14205